In [2]:
import requests
import pandas as pd

In [3]:
GL_QC_URL = 'https://raw.githubusercontent.com/ooi-integration/qc-lookup/master/data_qc_global_range_values.csv'
gl = pd.read_csv(GL_QC_URL)

In [4]:
# instance where recovered and telemtered parameter differ
gl[gl['ParameterID_R'] != gl['ParameterID_T']]

,ReferenceDesignator,ParameterID_R,ParameterID_T,GlobalRangeMin,GlobalRangeMax,_DataLevel,_Units,_Array ID,_Platform ID,_Instrument
2115,CP02PMUI-RII01-02-ADCPTG010,eastward_seawater_velocity,adcps_jln_eastward_seawater_velocity2,-3.0,3.0,L1,m s-1,CP02PMUI,RII01,ADCPTG010
2116,CP02PMUI-RII01-02-ADCPTG010,northward_seawater_velocity,adcps_jln_northward_seawater_velocity2,-3.0,3.0,L1,m s-1,CP02PMUI,RII01,ADCPTG010
2117,CP02PMUI-RII01-02-ADCPTG010,upward_seawater_velocity,adcps_jln_upward_seawater_velocity2,-1.0,1.0,L1,m s-1,CP02PMUI,RII01,ADCPTG010
2118,CP02PMUI-RII01-02-ADCPTG010,error_seawater_velocity,adcps_jln_error_seawater_velocity2,-1.0,1.0,L1,m s-1,CP02PMUI,RII01,ADCPTG010


In [5]:
# all parameters listed in qc sheet
p_r_gl = gl.ParameterID_R.values.tolist()
p_t_gl = gl.ParameterID_T.values.tolist()
p_gl = set(p_r_gl + p_t_gl)

In [6]:
# all parameters listed in preload
preload = 'https://raw.githubusercontent.com/oceanobservatories/preload-database/master/csv/ParameterDefs.csv'
pre_l = pd.read_csv(preload)
pre_l = pre_l.name.values.tolist()

In [7]:
# parameters listed in qc sheet that do not exist in preload
set(p_gl) - set(pre_l)

{'adcps_jln_error_seawater_velocity2'}

In [8]:
# compare to qc database output to global range sheets
kitchen_sink = 'inputs/unique_parameters.csv'
ks = pd.read_csv(kitchen_sink)
p_ks = set(ks.name.values.tolist())
set(p_gl) - set(p_ks)

{'adcpa_n_auv_error_velocity',
 'adcps_jln_error_seawater_velocity2',
 'average_direction',
 'vertical_velocity'}

filter qc database output

In [10]:
kitchen_sink = 'inputs/unique_parameters.csv'
ks = pd.read_csv(kitchen_sink)

# only select science streams
ks = ks[ks['stream_type'] == 'Science'] 

# delete stream columns
ks = ks.drop(columns=['stream_id',\
                      'time_parameter',\
                      'binsize_minutes',\
                      'stream_content',\
                      'stream_type',\
                      'stream_name',\
                      'description',\
                      'description.1']) 

# get unique parameter list
ks = ks.drop_duplicates() 

# remove parameters units that aren't run against QC values
ks = ks[(ks['unit'] != 'count') & (ks['unit'] != 'counts')]
ks = ks[ks['unit'] != 'volt']
ks = ks[ks['unit'] != '0%']
ks = ks[ks['unit'] != 'day']
ks = ks[ks['unit'] != 'bit_mask']
ks = ks[ks['unit'] != 'B']
ks = ks[ks['unit'] != 's']

# remove parameters with fill value 0
ks = ks[ks['fill_value'] != '0']

# # remove units 1, except if L1 or L2
ks = ks.drop(ks[(ks['unit'] == '1') & (ks['data_level'].isnull())].index)
ks = ks.drop(ks[(ks['unit'] == '1') & (ks['data_level'] == 0)].index)

# # remove units seconds since, except if it has a parameter function map
ks = ks.drop(ks[(ks['unit'].str.contains('seconds since')) & (ks['parameter_function_map'].isnull())].index)

In [29]:
p_ks = set(ks.name.values.tolist())
set(p_gl) - set(p_ks)
# f = list(set(p_gl) - set(p_ks))
# gl[gl['ParameterID_R'].isin(f)]

{'absorbance_blank_434',
 'absorbance_blank_620',
 'absorbance_ratio_434',
 'absorbance_ratio_620',
 'adcpa_n_auv_error_velocity',
 'adcps_jln_error_seawater_velocity2',
 'average_current',
 'average_depth',
 'average_direction',
 'beam_conf',
 'cal_press',
 'commanded_volume',
 'ctd_psu',
 'cumulative_volume',
 'date_time_array',
 'date_time_str',
 'dir_p2',
 'dofst_k_oxygen',
 'fdchp_speed_of_sound_sonic',
 'hpies_average_pressure',
 'hpies_average_temperature',
 'hpies_average_travel_time',
 'hpies_bliley_temperature',
 'hpies_compass_fwrite_attempted',
 'hpies_data_valid',
 'hpies_data_validity',
 'hpies_hcno',
 'hpies_hcno_last_cal',
 'hpies_hcno_last_comp',
 'hpies_ifok',
 'hpies_irq2_count',
 'hpies_last_pressure',
 'hpies_last_temperature',
 'hpies_mot_pwr_count',
 'hpies_n_travel_times',
 'hpies_pressure',
 'hpies_secs',
 'hpies_travel_time1',
 'hpies_travel_time2',
 'hpies_travel_time3',
 'hpies_travel_time4',
 'mag_comp_x',
 'mag_comp_y',
 'massp_converter_12v_main',
 'massp

In [31]:
f = list(set(p_ks) - set(p_gl))
ks[ks['name'].isin(f)]

,parameter_id,name,unit,fill_value,display_name,standard_name,parameter_precision,parameter_function_id,parameter_function_map,data_product_identifier,data_level,data_product_type,parameter_type,value_encoding
1145,2810.0,scaled_wave_burst_seafloor_pressure,dbar,-9999999,Seafloor Pressure,sea_water_pressure_at_sea_floor,4.0,132.0,"{""slope"": ""CC_slope_correction_factor"", ""ptcn""...",SFLPRES-WAVE_L1,1.0,Science Data,function,float32
1286,651.0,bin_1_distance,cm,65535,Bin 1 Distance,NaN,0.0,NaN,NaN,NaN,NaN,NaN,quantity,uint16
1288,671.0,transducer_depth,dm,65535,Transducer Depth,NaN,0.0,NaN,NaN,NaN,NaN,NaN,quantity,uint16
1289,672.0,heading,deci-degrees,65535,Heading,NaN,0.0,NaN,NaN,NaN,NaN,NaN,quantity,uint16
1290,673.0,pitch,deci-degrees,-9999,Pitch,NaN,0.0,NaN,NaN,NaN,NaN,NaN,quantity,int16
1291,674.0,roll,deci-degrees,-9999,Roll,NaN,0.0,NaN,NaN,NaN,NaN,NaN,quantity,int16
1307,729.0,percent_good_3beam,percent,255,Percent Good 3Beam,NaN,0.0,NaN,NaN,NaN,NaN,NaN,array<quantity>,uint8
1308,730.0,percent_transforms_reject,percent,255,Percent Transforms Reject,NaN,0.0,NaN,NaN,NaN,NaN,NaN,array<quantity>,uint8
1309,731.0,percent_bad_beams,percent,255,Percent Bad Beams,NaN,0.0,NaN,NaN,NaN,NaN,NaN,array<quantity>,uint8
1310,732.0,percent_good_4beam,percent,255,Percent Good 4Beam,NaN,0.0,NaN,NaN,NaN,NaN,NaN,array<quantity>,uint8
